In [1]:
import pandas as pd
import datetime
import re
import yfinance as yf
from io import StringIO


In [2]:
table_txt = """
	Nombre	Variación diaria	Último Operado AR$	Cantidad	Tenencia US$
	AR$	0.00%	1,00	284,46	0,28
	US$ MEP	0.00%	1.023,62	13.510,17	13.510,17
	AMZN	+5.09%	1.084,00	390,00	413,00
	MSFT	+5.54%	12.959,50	10,00	126,60
	SPY	+4.65%	24.632,50	1,00	24,06
	AL30	+3.55%	39.400,00	255,00	98,15
	GD30	+3.95%	41.580,00	200,00	81,24
"""

In [3]:
data = StringIO(table_txt)
df = pd.read_csv(data, delimiter='\t')
df.drop("Unnamed: 0", axis=1, inplace=True)
df

,Nombre,Variación diaria,Último Operado AR$,Cantidad,Tenencia US$
0,AR$,0.00%,"1,00","284,46","0,28"
1,US$ MEP,0.00%,"1.023,62","13.510,17","13.510,17"
2,AMZN,+5.09%,"1.084,00","390,00","413,00"
3,MSFT,+5.54%,"12.959,50","10,00","126,60"
4,SPY,+4.65%,"24.632,50","1,00","24,06"
5,AL30,+3.55%,"39.400,00","255,00","98,15"
6,GD30,+3.95%,"41.580,00","200,00","81,24"


In [4]:
def correct_str(input_str):
    input_str = input_str.replace(".", "")
    input_str = input_str.replace(",", ".")

    input_str = re.sub("[^0-9^.]", "", input_str)

    if input_str == '':
        input_str = 0.0
    return float(input_str)

In [5]:
for col in df.columns:
    if col != "Nombre":
        df[col] = df[col].apply(correct_str)

In [6]:
cedear_data = pd.read_excel("data/cedear_ratios_reloaded.xlsx", sheet_name=0)

In [7]:
df2 = df.merge(cedear_data, right_on="symbol", left_on="Nombre", how="left")

In [8]:
def find_price(stock_name):
    if stock_name is None:
        return 0.0
    else:
        return yf.download(stock_name, period="1d", interval="1d")["Adj Close"].iloc[0]

In [9]:
df2["base_symbol"] = df2["base_symbol"].astype(str)

In [10]:
df2["base_asset_price"] = df2["base_symbol"].apply(find_price)

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [11]:
df2["this_asset_price"] = df2["base_asset_price"] / df2["ratio"]

In [12]:
df2.set_index("Nombre", inplace=True)

In [18]:
today_str = datetime.date.today().strftime("%m/%d/%Y")
ccl_dollar = pd.read_csv("data/dolar_ccl_historic.csv", index_col=0)

In [19]:
df2.loc["AR$","this_asset_price"] = 1 / ccl_dollar.loc[today_str, "Referencia"]
df2.loc["US$ MEP","this_asset_price"] = 1


In [ ]:
for index, row in df2.iterrows():
    if pd.isna(row["this_asset_price"]):
        stock_name = index + "D.BA"
        try:
            price = yf.download(stock_name, period="1d", interval="1d")["Adj Close"].iloc[0]
            df2.loc[index,"this_asset_price"] = price
        except:
            pass


In [20]:
df2[["Cantidad", "this_asset_price"]]


,Cantidad,this_asset_price
Nombre,,
AR$,284.46,0.000974
US$ MEP,13510.17,1.000000
AMZN,390.00,1.034931
MSFT,10.00,12.387166
SPY,1.00,23.495000
AL30,255.00,NaN
GD30,200.00,NaN


In [ ]:
adsfasdf

In [ ]:
t ="""
DIA	Compra	US$ 18,25	38	US$ 693,50	48hs	Ejecutada
AAPL	Compra	US$ 19,30	37	US$ 714,10	48hs	Ejecutada
DIA	Compra	US$ 18,20	52	US$ 946,40	48hs	Ejecutada
DIA	Compra	US$ 18,20	2	US$ 36,40	48hs	Ejecutada
AAPL	Venta	US$ 19,55	84	US$ 1.642,20	48hs	Ejecutada
DIA	Venta	US$ 18,70	54	US$ 0,00	48hs	Cancelada
AMZN	Venta	US$ 1,05	724	US$ 760,20	48hs	Ejecutada
SPY	Venta	US$ 23,70	415	US$ 0,00	48hs	Cancelada
AMZN	Venta	US$ 1,05	724	US$ 0,00	48hs	Cancelada
AMZN	Compra	US$ 1,03	29	US$ 29,87	48hs	Ejecutada
NVDA	Venta	US$ 20,70	1	US$ 20,70	48hs	Ejecutada
SPY	Compra	US$ 23,35	10	US$ 233,50	48hs	Ejecutada
NVDA	Compra	US$ 20,45	9	US$ 0,00	48hs	Cancelada
NVDA	Compra	US$ 20,45	1	US$ 20,45	48hs	Ejecutada
NVDA	Venta	US$ 20,65	12	US$ 247,80	48hs	Ejecutada
NVDA	Compra	US$ 20,45	9	US$ 184,05	48hs	Ejecutada
NVDA	Venta	US$ 20,65	2	US$ 0,00	48hs	Cancelada
NVDA	Venta	US$ 20,65	1	US$ 20,65	48hs	Ejecutada
NVDA	Compra	US$ 20,45	4	US$ 81,80	48hs	Ejecutada
SPY	Compra	US$ 23,35	1	US$ 23,35	48hs	Ejecutada
NVDA	Venta	US$ 20,65	6	US$ 123,90	48hs	Ejecutada
AMZN	Compra	US$ 1,02	156	US$ 0,00	48hs	Cancelada
AMZN	Compra	US$ 1,02	15	US$ 15,30	CI	Ejecutada
NVDA	Venta	US$ 20,90	6	US$ 0,00	48hs	Cancelada
SPY	Venta	US$ 23,55	123	US$ 0,00	CI	Cancelada
SPY	Compra	US$ 23,35	33	US$ 770,55	48hs	Ejecutada
SPY	Compra	US$ 23,35	2	US$ 46,70	48hs	Ejecutada
SPY	Compra	US$ 23,35	2	US$ 46,70	48hs	Ejecutada
SPY	Compra	US$ 23,20	10	US$ 0,00	CI	Cancelada
CVX	Venta	US$ 19,00	2	US$ 38,00	48hs	Ejecutada
SPY	Venta	US$ 23,75	91	US$ 0,00	CI	Cancelada
AAPL	Venta	US$ 19,80	84	US$ 0,00	48hs	Cancelada
SPY	Venta	US$ 23,75	214	US$ 0,00	48hs	Cancelada
SPY	Compra	US$ 23,40	260	US$ 6.084,00	48hs	Ejecutada
AMZN	Venta	US$ 1,04	205	US$ 213,20	CI	Ejecutada
DIA	Venta	US$ 18,80	54	US$ 0,00	48hs	Cancelada
SPY	Venta	US$ 23,80	132	US$ 3.141,60	48hs	Ejecutada
CVX	Venta	US$ 19,90	2	US$ 0,00	48hs	Cancelada
SPY	Compra	US$ 23,30	63	US$ 1.467,90	CI	Ejecutada
"""

In [ ]:
data = StringIO(t)
df = pd.read_csv(data, delimiter='\t', header=None)
df

In [ ]:
df.columns = ["Especie", "Operación",   "Precio",  "Cantidad",    "Total operación",   "Plazo",   "Estado"]

In [ ]:
replacement_mapping = {'Compra': 1, 'Venta': -1}
df['Operación'] = df['Operación'].replace(replacement_mapping)

In [ ]:
df["Tot"] = df['Operación'] * df['Cantidad']

In [ ]:
df[df["Estado"] == "Ejecutada"].groupby("Especie").agg({"Tot": "sum"})

In [ ]:
df.dtypes